In [1]:
#Import required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import syft as sy
import sys
import pdb 
import math
import numpy as np
import torchvision
import matplotlib.pyplot as plt
from random import shuffle
from torch.utils.data import SubsetRandomSampler
import datetime
import copy
#from torch.utils.tensorboard import SummaryWriter

W0322 01:15:38.800972  5316 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'c:\users\florian\appdata\local\programs\python\python37\lib\site-packages\tf_encrypted-0.5.9-py3.7.egg\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.0-rc3.so'
W0322 01:15:38.827970  5316 module_wrapper.py:139] From c:\users\florian\appdata\local\programs\python\python37\lib\site-packages\tf_encrypted-0.5.9-py3.7.egg\tf_encrypted\session.py:24: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
#torch.set_default_tensor_type(torch.cuda.FloatTensor)
use_cuda = False
kwargs = {} if use_cuda else {}
device = torch.device("cuda" if use_cuda else "cpu")
np.random.seed(1)
torch.manual_seed(1)
torch.set_num_threads(4)

batch_size = 16
learning_rate = 0.0001
percentagePoisonedData = 0.25
NO_BENIGN = 4
NO_FRAUDS = 1
modelReplacement = True
backdoorType = "glasses"

# Creating workers

In [3]:
def createClients(num_training, num_frauds):
    """
    This function creates all data constracts necessary for the clients
    
    :param num_training: number of benign clients
    :param num_frauds: number of malicious clients
    
    :return remote_dataset: an empty construct of n-tuple of lists, where n equals the number of all clients 
    :return models: list containing models of benign and malicious clients
    :return params: list containing model parameters of benign and malicious clients
    :return optimizers: list containing model optimizers of benign and malicious clients
    :return compute_nodes: list containing all benign clients
    :return frauds: list containing all malicious clients
    """
    remote_dataset = tuple([list() for x in range((num_training + num_frauds))])
    models = []
    params = []
    optimizers = []
    compute_nodes = []
    frauds = []
    for i in range(num_training+num_frauds):
        m = Net().to(device)
        models.append(m)
        params.append(list(m.parameters()))
        optimizers.append(optim.Adam(m.parameters(), lr=learning_rate))
        
    for i in range(num_training):
        compute_nodes.append(sy.VirtualWorker(hook, id=("benign_" + str(i))))
    
    for i in range(num_frauds):
        frauds.append(sy.VirtualWorker(hook, id=("fraud_" + str(i))))
        
    return remote_dataset, models, params, optimizers, compute_nodes, frauds

# Loading training & test datasets

In [4]:
data_transform = transforms.Compose([
        transforms.Grayscale(1),
        transforms.Resize((64,64)),
        transforms.ToTensor(),                     
        transforms.Normalize((0.5,),(0.5,))])


#benign data
trafficsign = datasets.ImageFolder(root = 
                             'C:\\Users\\Florian\\Desktop\\yale_new\\original_extended',
                             transform=data_transform)
original_loader = torch.utils.data.DataLoader(trafficsign, 
                batch_size=batch_size,
                shuffle=True,
                **kwargs)

#benign test data
testdata = datasets.ImageFolder(root = 
                             'C:\\Users\\Florian\\Desktop\\yale_new\\original_test_extended',
                             transform=data_transform)

test_loader = torch.utils.data.DataLoader(testdata, batch_size=batch_size, **kwargs)



# Load backdoor dataset

In [5]:
#malicious data
path = 'C:\\Users\\Florian\\Desktop\\yale_new\\' + backdoorType + '_extended'
backdoored = datasets.ImageFolder(root = 
                             path,
                             transform=data_transform)
backdoored.samples = [(d, 0) for d, s in backdoored.samples] #set each image of backdoors to 001

backdoored_loader = torch.utils.data.DataLoader(backdoored, 
                batch_size=batch_size,
                shuffle=True,
                **kwargs)
#backdoored_loaders = generateLoadersPerClass(backdoored)

#malicious test data
path = 'C:\\Users\\Florian\\Desktop\\yale_new\\' + backdoorType + '_test_extended'
backdoored_test = datasets.ImageFolder(root = 
                             path,
                             transform=data_transform)
backdoored_test.samples = [(d, 0) for d, s in backdoored_test.samples] #set each image of backdoors to 001

dataset_loader_backdoored_test = torch.utils.data.DataLoader(backdoored_test, 
                                                             batch_size=batch_size, 
                                                             shuffle=True, **kwargs)

# Visualize some training data

In [6]:
class_names = trafficsign.classes

#Let’s visualize a few training images so as to understand the data augmentations.

#def imshow(inp, title=None):
#    """Imshow for Tensor."""
#    inp = inp.numpy().transpose((1, 2, 0))
#    mean = np.array([0.485, 0.456, 0.406])
#    std = np.array([0.229, 0.224, 0.225])
#    inp = std * inp + mean
#    inp = np.clip(inp, 0, 1)
#    plt.imshow(inp)
#    if title is not None:
#        plt.title(title)
#    plt.pause(0.001)  # pause a bit so that plots are updated
#
## Get a batch of training data
#inputs, classes = next(iter(dataset_loader_backdoored_test))
#
## Make a grid from batch
#out = torchvision.utils.make_grid(inputs)
#
#imshow(out, title=[class_names[x] for x in classes])

# Neural Network Structure

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv0 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=7) #kernel size = receptive field size
        self.bn0 = nn.BatchNorm2d(6) #use batchnorm for subsampling
        
        self.conv1 = nn.Conv2d(6, 16, 8)
        self.bn1 = nn.BatchNorm2d(16) #use batchnorm for subsampling

        self.fc0 = nn.Linear(16*51*51,15)            #First Fully-Connected Layer (256*12*12 for 64x64 images)


    def forward(self, x):
        #import pdb; pdb.set_trace()
        x = F.relu(self.bn0(self.conv0(x)))
        x = F.relu(self.bn1(self.conv1(x)))
        x = x.view(-1, 16*51*51)
        x = self.fc0(x)
        return F.log_softmax(x, dim=1)
    
#see for parameters: 
#https://discuss.pytorch.org/t/runtimeerror-shape-1-400-is-invalid-for-input-of-size/33354/5
#4- Batch size
#13 = squareRoot(10816/4/16)
#I had to set x = x.view(4, 16 * 13 * 13).


# Secure Multiparty computation: send datasets to clients

In [8]:
def distributeDataToClients(num_training, num_frauds, percentagePoisonedData):
    """
    This function distributes data loaded by the data loaders across the clients
    
    :param num_training: number of benign clients
    :param num_frauds: number of malicious clients
    :param percentagePoisonedData: the relation of poisoned data to non-poisoned data in malicious clients, range 0-1
    """
    global remote_dataset
    total_number_of_clients = num_training + num_frauds
    total_batches = 0
    #add original data
    for i, (d,t) in enumerate(original_loader): # run multiple times over the dataset to increase its size
        data = d.to(device)
        target = t.to(device)
        if(percentagePoisonedData == 1.0): #do not add original data to backdoor clients
            data = data.send(compute_nodes[i % num_training])
            target = target.send(compute_nodes[i % num_training])
            remote_dataset[i % num_training].append((data, target))
        else: #also add original data to backdoor clients (and remove after)
            targetClient = i % total_number_of_clients
            if (targetClient < num_training): #add to benign clients
                data = data.send(compute_nodes[targetClient])
                target = target.send(compute_nodes[targetClient])
            else: #add to malicious clients
                data = data.send(frauds[targetClient-num_training])
                target = target.send(frauds[targetClient-num_training])
            remote_dataset[targetClient].append((data, target))
        total_batches += 1
    

    if (num_frauds != 0):
        all_backdoored_data_dict = {}
        #get subset of data to match with the number of benign and malicious nodes (100% backdoor, 0% benign data)
        total_data = total_batches * (len(compute_nodes) + len(frauds))/len(compute_nodes)
        fraction_of_backdoored_clients = len(frauds)/(len(compute_nodes) + len(frauds))
        numberOfBatchesForBackdoor = int(total_data*fraction_of_backdoored_clients/len(frauds)*percentagePoisonedData)
                                         
        #add list to dict for each fraud
        for f in frauds:
            all_backdoored_data_dict[f] = []
        
        for i, (d,t) in enumerate(backdoored_loader):
            #add all backdoor data
            data = d.to(device)
            target = t.to(device)
            data = data.send(frauds[(i+1) % len(frauds)])
            target = target.send(frauds[(i+1) % len(frauds)])
            all_backdoored_data_dict.get(frauds[(i+1) % len(frauds)]).append((data, target))
        

        #shorten benign data
        for i in range(num_frauds): 
            shuffle(remote_dataset[num_training+i])
            temp = list(remote_dataset)
            temp[num_training+i] = temp[num_training+i][:(len(temp[0])-numberOfBatchesForBackdoor)]
            remote_dataset = tuple(temp)
        
        #add each backdoor to remote_dataset
        for x in all_backdoored_data_dict:
            length = 0
            for (d,t) in all_backdoored_data_dict[x]: 
                if(length < numberOfBatchesForBackdoor):
                    remote_dataset[num_training + frauds.index(x)].append((d, t)) # append new backdoored batch
                    length += 1

In [ ]:
hook = sy.TorchHook(torch)

remote_dataset, models, params, optimizers, compute_nodes, frauds = createClients(NO_BENIGN,NO_FRAUDS)
distributeDataToClients(NO_BENIGN,NO_FRAUDS,percentagePoisonedData)

def update(data, target, model, optimizer):
    """
    This function updates the current neural network
    
    :param data: batch of data
    :param target: batch of target classes
    :param model: the model being updated
    :param optimizer: the model's optimizer
    
    :return losscopy: the current value of the lossfunction
    :return model: the updated model
    """
    model.send(data.location)
    optimizer.zero_grad()
    pred = model(data)
    loss = F.cross_entropy(pred, target)
    losscopy = float(loss.copy().get())
    loss.backward()
    optimizer.step()
    del loss
    del pred
    return losscopy, model

def update_poison(data, target, model, optimizer):
    """
    This function updates the current neural network following the model replacement attack 
    (see https://arxiv.org/pdf/1807.00459.pdf)
    
    :param data: batch of data
    :param target: batch of target classes
    :param model: the model being updated
    :param optimizer: the model's optimizer
    
    :return losscopy: the current value of the lossfunction
    :return updated_Model: the updated model
    """
    clip_rate = int(len(compute_nodes) + len(frauds)) #number_of_participants
    oldModel = copy.deepcopy(model).send(data.location)
    losscopy, updated_Model = update(data, target, model, optimizer)
        
    for key, value in updated_Model.state_dict().items():
        old_value = oldModel.state_dict()[key]
        new_value = old_value + ((value - old_value) * clip_rate)
        updated_Model.state_dict()[key].copy_(new_value)
    del oldModel
    return losscopy, updated_Model

# Training Function

In [ ]:
def trainSMPC(epoch, modelReplacement):
    """
    This function trains the global model by averaging local trained models.
    
    :param epoch: integer of current training epoch
    :param modelReplacement: boolean if model replacement method should be used or not
    
    :return total_mean_loss: mean of all losses of all benign clients in current epoch
    :return total_mean_loss_backdoor: mean of all losses of all malicious clients in current epoch
    """

    minimumOfBatchesOverAllClients = min(map(len, remote_dataset)) #length of smallest list of clientdata- ensures that all clients participate each iteration 
    total_number_of_clients = int(len(compute_nodes) + len(frauds))
    total_mean_loss = 0
    total_mean_loss_backdoor = 0
    for data_index in range(minimumOfBatchesOverAllClients): #iterates over batches
        mean_loss = 0
        mean_loss_backdoor = 0
        print(f"update remote models {data_index+1} / {minimumOfBatchesOverAllClients}")
        for remote_index in range(total_number_of_clients): #each client of a batch
            d, t = remote_dataset[remote_index][data_index]
            data = d.to(device)
            target = t.to(device)
            del d
            del t
            
            if(modelReplacement == True):
                if (remote_index > len(compute_nodes)-1):
                    losscopy_backdoor, model = update_poison(data, target, models[remote_index], optimizers[remote_index])
                    models[remote_index] = model
                    mean_loss_backdoor = losscopy_backdoor
                else:
                    losscopy, model = update(data, target, models[remote_index], optimizers[remote_index])
                    models[remote_index] = model
                    mean_loss += losscopy
            else:
                losscopy, model = update(data, target, models[remote_index], optimizers[remote_index])
                models[remote_index] = model
                mean_loss += losscopy
                if(remote_index > len(compute_nodes)-1):
                    mean_loss_backdoor+= losscopy

        
        new_params = list()
        for param_i in range(len(params[0])): #for each parameter
            spdz_params = list()
            for remote_index in range(total_number_of_clients): #for each client
                copy_of_parameter = (params[remote_index][param_i]).get()
                spdz_params.append(copy_of_parameter)
            
            new_param = sum(spdz_params)/total_number_of_clients
            new_params.append(new_param)
        
        with torch.no_grad():
            for m in params:
                for param in m:
                    param *= 0

            for remote_index in range(total_number_of_clients):
                for param_index in range(len(params[remote_index])):
                    params[remote_index][param_index].data = new_params[param_index]
        
        #delete stuff
        del new_params
        del spdz_params
                    
        total_mean_loss += (mean_loss/len(compute_nodes))
        if(len(frauds) != 0):
            total_mean_loss_backdoor += (mean_loss_backdoor/len(frauds))
            
    total_mean_loss = (total_mean_loss/minimumOfBatchesOverAllClients)
    total_mean_loss_backdoor = (total_mean_loss_backdoor/minimumOfBatchesOverAllClients)
    return total_mean_loss, total_mean_loss_backdoor

In [ ]:
def test(model, device, test_loader, length_of_dataset):
    model.eval()
    test_loss = 0
    correct = 0    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= length_of_dataset

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, length_of_dataset,
        100. * correct / length_of_dataset))
    
    #confusion matrix
    nb_classes = len(class_names)
    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    with torch.no_grad():
        for i, (inputs, classes) in enumerate(test_loader):
            inputs = inputs.to(device)
            classes = classes.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            for t, p in zip(classes.view(-1), preds.view(-1)):
                    confusion_matrix[t.long(), p.long()] += 1
    #print(confusion_matrix)
    per_class_accuracy = confusion_matrix.diag()/confusion_matrix.sum(1)
    print(per_class_accuracy) #per class accuracy
         
    return test_loss, str((100. * correct / length_of_dataset)), per_class_accuracy

# Run everyting

In [ ]:
#Write to file:
dateString = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

f= open(("exp_yale_"+dateString+".txt"),"w+")

#EXP-setup
csv_header =  "#dataset: " + "yale" + "\n"
csv_header += "#way backdoor looks like: " + backdoorType + "\n"
csv_header += "#merge strategy: " + "aggregation" + "\n"
csv_header += "#learning rate: " + str(learning_rate) + "\n"
csv_header += "#number of benign sources: " + str(len(compute_nodes)) + "\n"
csv_header += "#number of malicious sources: " + str(len(frauds)) + "\n"
csv_header += "#batch size: " + str(batch_size) + "\n"
csv_header += "#distribution of data: " + "equally distributed subset" + "\n"
csv_header += "#percentage of poisoned data in backdoored nodes: " + str(percentagePoisonedData) + "\n" #str(100)
csv_header += "#order of time backdoors being inserted: " + "no" + "\n" #backdoors first
csv_header += "#model replacement: " + str(modelReplacement) + "\n"
csv_header += "#starttime: " + datetime.datetime.now().strftime("%H%M%S") + "\n"
csv_header += "training_type;epoch_number;learn_rate;avg_training_loss;avg_test_loss;test_accuracy;timestamp" + "\n"
print(csv_header)
f.write(csv_header)
f.close()


#RUN training
for epoch in range(1,101):
    print(f"Epoch {epoch}")
    
    csv_normal = "normal;" + str(epoch) + ";" + str(learning_rate) + ";"
    csv_backdoor = "backdoor;" + str(epoch) + ";" + str(learning_rate) + ";"

    #train both - set order in method
    avg_training_loss, avg_training_backdoor_loss = trainSMPC(epoch, modelReplacement)
    csv_normal += str(avg_training_loss) + ";"
    csv_backdoor += str(avg_training_backdoor_loss) + ";"
    timestamp_combined = datetime.datetime.now().strftime("%H%M%S")
    
    #save after each 10 iterations
    #if epoch % 99 == 0:
    #    torch.save(models[0].state_dict(), ("exp_yale_"+dateString +"_epoch_" + str(epoch) + ".pt"))
    
    #test backdoor
    test_loss, acc, per_class_accuracyBackdoor = test(models[0], device, dataset_loader_backdoored_test, len(backdoored_test))
    csv_backdoor += str(test_loss) + ";" + acc + ";"
    
    #test normal
    test_loss, acc, per_class_accuracy = test(models[0], device, test_loader, len(testdata))
    csv_normal += str(test_loss) + ";" + acc + ";"
    
    #timestamps
    csv_normal += timestamp_combined + "\n"
    csv_backdoor += timestamp_combined + "\n"
    
    #Write to file
    f= open(("exp_yale_"+dateString+".txt"),"a+")
    f2= open(("exp_yale_perClassAccuracy_"+dateString+".txt"),"a+")

    f.write(csv_backdoor)
    f.write(csv_normal)
    f2.write(str(per_class_accuracy) + "\n")
    
    f.close()
    f2.close()


#dataset: yale
#way backdoor looks like: glasses
#merge strategy: aggregation
#learning rate: 0.0001
#number of benign sources: 4
#number of malicious sources: 1
#batch size: 16
#distribution of data: equally distributed subset
#percentage of poisoned data in backdoored nodes: 0.25
#order of time backdoors being inserted: no
#model replacement: True
#starttime: 011552
training_type;epoch_number;learn_rate;avg_training_loss;avg_test_loss;test_accuracy;timestamp

Epoch 1
update remote models 1 / 17
update remote models 2 / 17
update remote models 3 / 17
update remote models 4 / 17
update remote models 5 / 17
update remote models 6 / 17
update remote models 7 / 17
update remote models 8 / 17
update remote models 9 / 17
update remote models 10 / 17
update remote models 11 / 17
update remote models 12 / 17
update remote models 13 / 17
update remote models 14 / 17
update remote models 15 / 17
update remote models 16 / 17
update remote models 17 / 17

Test set: Average loss: 2.6314, Accuracy:


tensor([1.0000, 0.9000, 0.8500, 0.6000, 0.8000, 0.5000, 0.9000, 0.6500, 0.9500,
        0.5000, 1.0000, 0.5500, 1.0000, 0.6500, 0.6500])
Epoch 10
update remote models 1 / 17
update remote models 2 / 17
update remote models 3 / 17
update remote models 4 / 17
update remote models 5 / 17
update remote models 6 / 17
update remote models 7 / 17
update remote models 8 / 17
update remote models 9 / 17
update remote models 10 / 17
update remote models 11 / 17
update remote models 12 / 17
update remote models 13 / 17
update remote models 14 / 17
update remote models 15 / 17
update remote models 16 / 17
update remote models 17 / 17

Test set: Average loss: 2.0460, Accuracy: 46/170 (27%)

tensor([0.2706,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan])

Test set: Average loss: 0.7502, Accuracy: 229/300 (76%)

tensor([1.0000, 0.8500, 0.8000, 0.6000, 0.8000, 0.5000, 0.9000, 0.7000, 0.9500,
        0.5000, 1.0000, 0.5500, 1.000

In [ ]:
torch.save(models[0].state_dict(), ("exp_yale_"+dateString +"_epoch_" + str(epoch) + ".pt"))

In [ ]:
#test model parameter
#m = Net().to(device)
#tempmodel = m.load_state_dict(torch.load("exp_traffic_20191217-164024_epoch_30.pt"))
#list(m.parameters())[8]